[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1F6rZjsXF6Mn_BhVIyc6Z2ph13KjbajaY?usp=sharing)

Source: https://github.com/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb

# Finetuning and feature extractor using HuggingFace models

This notebook is an updated version of the notebook at the following address.

https://github.com/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb


We will follow the steps of the notebook and make a possible alternative to the network-wide finetuning known as feature extractor or transfer-learning.

If you're opening this Notebook on colab, you will probably need to install the most recent versions of 🤗 Transformers and 🤗 Datasets. We will also need `scipy` and `scikit-learn` for some of the metrics. Uncomment the following cell and run it.

In [ ]:
! pip -q install transformers
! pip -q install datasets evaluate
! pip -q install scipy sklearn

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).


The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use Keras to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (a task with the same training set as `mnli` but different validation and test sets):

In [ ]:
GLUE_TASKS = [
    "cola",
    "mnli",
    "mnli-mm",
    "mrpc",
    "qnli",
    "qqp",
    "rte",
    "sst2",
    "stsb",
    "wnli",
]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set these three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and the [🤗 Evaluate](https://github.com/huggingface/datasets) library to get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the `load_dataset` function from `datasets` and and the `load` function from `evaluate`.

In [ ]:
from datasets import load_dataset
from evaluate import load

With the exception of `mnli-mm`, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load("glue", actual_task)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,Jean reads never this newspaper.,unacceptable,1080
1,It would be inconvenience for you to leave so soon.,acceptable,5040
2,They believe there to be a dragon in the wood.,acceptable,4626
3,I ran a man who was old down.,unacceptable,1109
4,It tried to bother me that Chris lied.,unacceptable,4284
5,To whom do you think it is the book that Mary gave?,unacceptable,5149
6,Why do you think he left?,acceptable,4843
7,The Emperor's every wish was immediately carried out.,acceptable,7895
8,John rolled the ball from the tree to the bush.,acceptable,655
9,"Him, they let him go yesterday.",acceptable,1853


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': -0.09416472308615167}

Note that `load` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2003, 1037, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the arguments `truncation=True` and `padding='longest`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model, and all inputs will be padded to the maximum input length to give us a single input array. A more performant method that reduces the number of padding tokens is to write a generator or `tf.data.Dataset` to only pad each *batch* to the maximum length in that batch, but most GLUE tasks are relatively quick on modern GPUs either way.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset["train"][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Columns added by tokenizer: ['attention_mask', 'input_ids']


In [ ]:
encoded_dataset["train"].features["label"]

ClassLabel(names=['unacceptable', 'acceptable'], id=None)

## Fine-tuning

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `TFAutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels).

In [ ]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

num_labels = 3 if task.startswith("mnli") else 1 if task == "stsb" else 2
if task == "stsb":
    num_labels = 1
elif task.startswith("mnli"):
    num_labels = 3
else:
    num_labels = 2

# This next little bit is optional, but will give us cleaner label outputs later
# If you're using a task other than CoLA, you will probably need to change these
# to match the label names for your task!
id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Obtén los pesos de todas las capas del modelo
bert_layer_weights = model.layers[0].get_weights()
pc_layer_weights = model.layers[0].get_weights()

print("Pesos de las primeras capas BERT:")
for layer_idx in range(10):
    print(f"Capa {layer_idx + 1}: {bert_layer_weights[layer_idx].shape}")

print("\nPesos de las capas del pre-classifer (capas ocultas):")
for layer_idx in range(2):
    print(f"Capa {layer_idx + 1}: {pc_layer_weights[layer_idx].shape}")

# Puedes acceder a pesos específicos de capas individuales si lo deseas
# Por ejemplo, los pesos de la primera capa
print()
first_layer_weights = bert_layer_weights[0]
first_layer_weights

Pesos de las primeras capas BERT:
Capa 1: (30522, 768)
Capa 2: (512, 768)
Capa 3: (768,)
Capa 4: (768,)
Capa 5: (768, 768)
Capa 6: (768,)
Capa 7: (768, 768)
Capa 8: (768,)
Capa 9: (768, 768)
Capa 10: (768,)

Pesos de las capas del pre-classifer (capas ocultas):
Capa 1: (30522, 768)
Capa 2: (512, 768)



array([[-0.01664949, -0.06661227, -0.01632868, ..., -0.01999032,
        -0.05139988, -0.0263568 ],
       [-0.01319846, -0.06733431, -0.01605646, ..., -0.0226614 ,
        -0.05537301, -0.02600443],
       [-0.01759106, -0.07094341, -0.01443494, ..., -0.02457913,
        -0.05956192, -0.0231829 ],
       ...,
       [-0.0231029 , -0.05878259, -0.01048967, ..., -0.01945743,
        -0.02615411, -0.02118432],
       [-0.0490171 , -0.05614787, -0.00465348, ..., -0.01065376,
        -0.01797333, -0.02187675],
       [-0.00646111, -0.0914881 , -0.00254872, ..., -0.01505679,
        -0.05040044,  0.04597744]], dtype=float32)

In [ ]:
validation_key = (
    "validation_mismatched"
    if task == "mnli-mm"
    else "validation_matched"
    if task == "mnli"
    else "validation"
)

tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset[validation_key],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import create_optimizer

num_epochs = 1
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

With that out of the way, how do we actually use `KerasMetricCallback`? It's straightfoward: We simply define a function that computes metrics given a tuple of numpy arrays of predictions and labels, then we pass that, along with the validation set to compute metrics on, to the callback:

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback


def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

callbacks = [metric_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
)

534/534 [==============================] - 85s 109ms/step - loss: 0.5179 - val_loss: 0.4737 - matthews_correlation: 0.4587


In [ ]:
# Comprobamos que efectivamente tanto las capas del clasificador com olos pesos del transformer han cambiado
bert_layer_weights_ft = model.layers[0].get_weights()
pc_layer_weights_ft = model.layers[1].get_weights()

print("Pesos de las primeras capas BERT:")
for layer_idx in range(10):
    print(f"Capa {layer_idx + 1}: {bert_layer_weights_ft[layer_idx].shape}")
    same_weights = np.array_equal(bert_layer_weights_ft[layer_idx], bert_layer_weights[layer_idx])
    print(f"Capas iguales al checkpoint inicial: {same_weights}")

print("\nPesos de las capas del pre-classifer (capas ocultas):")
for layer_idx in range(2):
    print(f"Capa {layer_idx + 1}: {pc_layer_weights_ft[layer_idx].shape}")
    same_weights = np.array_equal(pc_layer_weights_ft[layer_idx], pc_layer_weights[layer_idx])
    print(f"Capas iguales al checkpoint inicial: {same_weights}")

Pesos de las primeras capas BERT:
Capa 1: (30522, 768)
Capas iguales al checkpoint inicial: False
Capa 2: (512, 768)
Capas iguales al checkpoint inicial: False
Capa 3: (768,)
Capas iguales al checkpoint inicial: False
Capa 4: (768,)
Capas iguales al checkpoint inicial: False
Capa 5: (768, 768)
Capas iguales al checkpoint inicial: False
Capa 6: (768,)
Capas iguales al checkpoint inicial: False
Capa 7: (768, 768)
Capas iguales al checkpoint inicial: False
Capa 8: (768,)
Capas iguales al checkpoint inicial: False
Capa 9: (768, 768)
Capas iguales al checkpoint inicial: False
Capa 10: (768,)
Capas iguales al checkpoint inicial: False

Pesos de las capas del pre-classifer (capas ocultas):
Capa 1: (768, 768)
Capas iguales al checkpoint inicial: False
Capa 2: (768,)
Capas iguales al checkpoint inicial: False


## Fine-tuning only head layer

In this case we just want to use the pre-trained embedding as an extractor feature and apply dense layers at the end of the network to suit our problem. Let's see how it would change our problem.

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Congelar pesos

# en tensorflow
model.distilbert.trainable = False
model.summary()

# # en pytorch
# for param in model.distilbert.parameters():
#     param.requires_grad = False

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 592,130
Non-trainable params: 66,362,880
_________________________________________________________________


In [ ]:
from transformers import create_optimizer

num_epochs = 1
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

callbacks = [metric_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
)

534/534 [==============================] - 30s 43ms/step - loss: 0.6059 - val_loss: 0.6112 - matthews_correlation: 0.0000e+00


In [ ]:
bert_layer_weights_ft = model.layers[0].get_weights()
pc_layer_weights_ft = model.layers[1].get_weights()

print("Pesos de las primeras capas BERT:")
for layer_idx in range(10):
    print(f"Capa {layer_idx + 1}: {bert_layer_weights_ft[layer_idx].shape}")
    same_weights = np.array_equal(bert_layer_weights_ft[layer_idx], bert_layer_weights[layer_idx])
    print(f"Capas iguales al checkpoint inicial: {same_weights}")

print("\nPesos de las capas del pre-classifer (capas ocultas):")
for layer_idx in range(2):
    print(f"Capa {layer_idx + 1}: {pc_layer_weights_ft[layer_idx].shape}")
    same_weights = np.array_equal(pc_layer_weights_ft[layer_idx], pc_layer_weights[layer_idx])
    print(f"Capas iguales al checkpoint inicial: {same_weights}")

Pesos de las primeras capas BERT:
Capa 1: (30522, 768)
Capas iguales al checkpoint inicial: True
Capa 2: (512, 768)
Capas iguales al checkpoint inicial: True
Capa 3: (768,)
Capas iguales al checkpoint inicial: True
Capa 4: (768,)
Capas iguales al checkpoint inicial: True
Capa 5: (768, 768)
Capas iguales al checkpoint inicial: True
Capa 6: (768,)
Capas iguales al checkpoint inicial: True
Capa 7: (768, 768)
Capas iguales al checkpoint inicial: True
Capa 8: (768,)
Capas iguales al checkpoint inicial: True
Capa 9: (768, 768)
Capas iguales al checkpoint inicial: True
Capa 10: (768,)
Capas iguales al checkpoint inicial: True

Pesos de las capas del pre-classifer (capas ocultas):
Capa 1: (768, 768)
Capas iguales al checkpoint inicial: False
Capa 2: (768,)
Capas iguales al checkpoint inicial: False
